In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Attack_Prediction.csv')

In [2]:
columns=df.columns

for column in columns:
    if df[column].dtype=="int32":
        df[column]=df[column].astype("int16")
    elif df[column].dtype=="float64":
        df[column]=df[column].astype("float16")
    elif df[column].dtype=="object":
        df[column]=df[column].astype("category")

In [3]:
df['Sex']= df['Sex'].map({'Female': 0, 'Male': 1})
df['Sex']= pd.to_numeric(df['Sex'])

df['Diet']= df['Diet'].map({'Healthy': 0, 'Average': 1, 'Unhealthy':2})
df['Diet']= pd.to_numeric(df['Diet'])

df[['HBP', 'LBP']]= df['Blood Pressure'].str.split('/', expand= True)
df['HBP']= pd.to_numeric(df['HBP'])
df['LBP']= pd.to_numeric(df['LBP'])

df['Diabetes'] = df['Diabetes'].map({0: 1, 1: 0})

df['Exercise Hours Per Week']= round(df['Exercise Hours Per Week'], 0)

df['Sedentary Hours Per Day']= round(df['Sedentary Hours Per Day'], 0)

df['Income']= round(df['Income'], 0)

df['BMI']= round(df['BMI'], 0)

df = df.drop(columns=['Patient ID', 'Blood Pressure', 'Country', 'Continent', 'Hemisphere'])

In [4]:
X= df.drop(['Heart Attack Risk'], axis= 1)
y= df['Heart Attack Risk']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [6]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [7]:
def backward_elimination(data, target, significance_level= 0.05):
    features= data.columns.tolist()
    while len(features) > 0:
        model= sm.Logit(target, data[features])
        result= model.fit(disp=0)
        p_values= result.pvalues
        max_p_value= p_values.max()
        if max_p_value > significance_level:
            excluded_feature= p_values.idxmax()
            print(f"Excluding {excluded_feature} with p-value {max_p_value}")
            features.remove(excluded_feature)
        else:
            break
    return data[features]

In [8]:
import statsmodels.api as sm

X_sm_tl_df= pd.DataFrame(X_sm_tl, columns= X.columns)
X_test_df= pd.DataFrame(X_test, columns= X.columns)

selected_features= backward_elimination(X_sm_tl_df, y_sm_tl)
X_sm_tl_selected= X_sm_tl_df[selected_features.columns]
X_sm_tl_test_selected= X_test_df[selected_features.columns]

Excluding Previous Heart Problems with p-value 0.5922424503542401
Excluding Heart Rate with p-value 0.5375402544518231
Excluding Medication Use with p-value 0.5371949883241286
Excluding Exercise Hours Per Week with p-value 0.46457319350133663
Excluding BMI with p-value 0.37186624629935927
Excluding Income with p-value 0.22276786932069725
Excluding Family History with p-value 0.18892766009225215
Excluding LBP with p-value 0.13510448212498058
Excluding Stress Level with p-value 0.12072913727436829
Excluding Diabetes with p-value 0.1060639301395154
Excluding Alcohol Consumption with p-value 0.056935231207391496
Excluding Triglycerides with p-value 0.05579199119392863
Excluding Diet with p-value 0.051267855262461505


--- LogisticRegression ---

In [9]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.63      0.49      0.55      1125
           1       0.35      0.49      0.41       628

    accuracy                           0.49      1753
   macro avg       0.49      0.49      0.48      1753
weighted avg       0.53      0.49      0.50      1753

Precision: 0.3503981797497156
Accuracy: 0.4917284654877353
AUC: 0.48647416843595187


--- DecisionTreeClassifier ---

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.64      0.62      0.63      1125
           1       0.35      0.37      0.36       628

    accuracy                           0.53      1753
   macro avg       0.50      0.50      0.50      1753
weighted avg       0.54      0.53      0.53      1753

Precision: 0.35454545454545455
Accuracy: 0.5322304620650313
AUC: 0.4969723991507431


--- Tuned - DecisionTreeClassifier ---

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 2}
Best Precision Score: 0.6250674694419338


In [12]:
tuned_dt= gs_dt.best_estimator_
tuned_dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.64      0.93      0.76      1125
           1       0.38      0.07      0.12       628

    accuracy                           0.63      1753
   macro avg       0.51      0.50      0.44      1753
weighted avg       0.55      0.63      0.53      1753

Precision: 0.38333333333333336
Accuracy: 0.6257843696520251
AUC: 0.5122540693559803


--- KNeighborsClassifier ---

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.64      0.54      0.59      1125
           1       0.36      0.46      0.40       628

    accuracy                           0.51      1753
   macro avg       0.50      0.50      0.49      1753
weighted avg       0.54      0.51      0.52      1753

Precision: 0.3588162762022195
Accuracy: 0.511123787792356
AUC: 0.4987692852087756


--- Tuned - KNeighborsClassifier ---

In [14]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.7348603789509164


In [15]:
tuned_knn= gs_knn.best_estimator_
tuned_knn.fit(X_sm_tl_selected, y_sm_tl)

y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.64      0.80      0.71      1125
           1       0.33      0.18      0.23       628

    accuracy                           0.58      1753
   macro avg       0.48      0.49      0.47      1753
weighted avg       0.53      0.58      0.54      1753

Precision: 0.3323442136498516
Accuracy: 0.5772960638904735
AUC: 0.4636093418259023


--- GaussianNB ---

In [16]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.63      0.51      0.56      1125
           1       0.34      0.46      0.39       628

    accuracy                           0.49      1753
   macro avg       0.49      0.48      0.48      1753
weighted avg       0.53      0.49      0.50      1753

Precision: 0.34330143540669855
Accuracy: 0.49229891614375354
AUC: 0.4808535031847133


--- SVM ---

In [17]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.63      0.62      0.63      1125
           1       0.34      0.34      0.34       628

    accuracy                           0.52      1753
   macro avg       0.48      0.48      0.48      1753
weighted avg       0.53      0.52      0.52      1753

Precision: 0.33751962323390894
Accuracy: 0.5236737022247575
AUC: 0.48216348195329084


--- Random Forest ---

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [19]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.64      0.81      0.72      1125
           1       0.37      0.20      0.26       628

    accuracy                           0.59      1753
   macro avg       0.51      0.51      0.49      1753
weighted avg       0.55      0.59      0.55      1753

Precision: 0.3699421965317919
Accuracy: 0.5904164289788933
AUC: 0.5008577494692144


In [20]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.8523196063573331


In [21]:
tuned_rf= gs_tuned_rf.best_estimator_
tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.64      0.82      0.72      1125
           1       0.34      0.17      0.22       628

    accuracy                           0.59      1753
   macro avg       0.49      0.49      0.47      1753
weighted avg       0.53      0.59      0.54      1753

Precision: 0.34201954397394135
Accuracy: 0.5864232743867656
AUC: 0.4999058740268931


--- AdaBoost ---

In [22]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [23]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.63      0.64      0.64      1125
           1       0.33      0.32      0.32       628

    accuracy                           0.53      1753
   macro avg       0.48      0.48      0.48      1753
weighted avg       0.52      0.53      0.52      1753

Precision: 0.33166666666666667
Accuracy: 0.5265259555048488
AUC: 0.48171196036801134


In [24]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.6590501578274092


In [25]:
tuned_ada= gs_ada.best_estimator_
tuned_ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.64      0.94      0.76      1125
           1       0.29      0.05      0.08       628

    accuracy                           0.62      1753
   macro avg       0.46      0.49      0.42      1753
weighted avg       0.51      0.62      0.52      1753

Precision: 0.2871287128712871
Accuracy: 0.6172276098117513
AUC: 0.49564614295824483


--- GradientBoosting ---

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.64      0.98      0.77      1125
           1       0.36      0.02      0.04       628

    accuracy                           0.64      1753
   macro avg       0.50      0.50      0.41      1753
weighted avg       0.54      0.64      0.51      1753

Precision: 0.358974358974359
Accuracy: 0.6354820308043354
AUC: 0.5065364472753008


In [27]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.7257639977763283


In [28]:
tuned_grb= gs_grb.best_estimator_
tuned_grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.65      0.95      0.77      1125
           1       0.42      0.06      0.11       628

    accuracy                           0.63      1753
   macro avg       0.53      0.51      0.44      1753
weighted avg       0.57      0.63      0.53      1753

Precision: 0.4222222222222222
Accuracy: 0.6337706788362807
AUC: 0.5047402689313517


--- XGB ---

In [29]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.65      0.81      0.72      1125
           1       0.40      0.22      0.28       628

    accuracy                           0.60      1753
   macro avg       0.52      0.52      0.50      1753
weighted avg       0.56      0.60      0.57      1753

Precision: 0.3954154727793696
Accuracy: 0.6001140901312036
AUC: 0.524966737438075


In [30]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
tuned_xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.1}
Best Precision Score: 0.7257272612827685
              precision    recall  f1-score   support

           0       0.65      0.90      0.75      1125
           1       0.39      0.12      0.18       628

    accuracy                           0.62      1753
   macro avg       0.52      0.51      0.46      1753
weighted avg       0.55      0.62      0.55      1753

Precision: 0.3862433862433862
Accuracy: 0.6172276098117513
AUC: 0.511660297239915


--- LGBM ---

In [31]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000924 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1878
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.64      0.90      0.75      1125
           1       0.38      0.11      0.17       628

    accuracy                           0.62      1753
   macro avg       0.51      0.50      0.46      1753
weighted avg       0.55      0.62      0.54      1753

Precision: 0.37988826815642457
Accuracy: 0.6172276098117513
AUC: 0.5089469214437368


In [32]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
tuned_lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 4288, number of negative: 4288
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000846 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4911
[LightGBM] [Info] Number of data points in the train set: 8576, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.01, 'min_data_in_leaf': 30, 'num_leaves': 31, 'reg_alpha': 0.1}
Best Precision Score: 0.8482220142848164
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_le

--- CatBoost ---

In [33]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.02579
0:	learn: 0.6898382	total: 153ms	remaining: 2m 33s
1:	learn: 0.6884103	total: 158ms	remaining: 1m 18s
2:	learn: 0.6849247	total: 162ms	remaining: 53.8s
3:	learn: 0.6823660	total: 166ms	remaining: 41.4s
4:	learn: 0.6780023	total: 171ms	remaining: 33.9s
5:	learn: 0.6761814	total: 176ms	remaining: 29.1s
6:	learn: 0.6732349	total: 180ms	remaining: 25.6s
7:	learn: 0.6704801	total: 186ms	remaining: 23s
8:	learn: 0.6682300	total: 191ms	remaining: 21s
9:	learn: 0.6655569	total: 195ms	remaining: 19.3s
10:	learn: 0.6642823	total: 199ms	remaining: 17.9s
11:	learn: 0.6617805	total: 204ms	remaining: 16.8s
12:	learn: 0.6596119	total: 208ms	remaining: 15.8s
13:	learn: 0.6575152	total: 212ms	remaining: 14.9s
14:	learn: 0.6554066	total: 216ms	remaining: 14.2s
15:	learn: 0.6540004	total: 220ms	remaining: 13.6s
16:	learn: 0.6503981	total: 225ms	remaining: 13s
17:	learn: 0.6489621	total: 229ms	remaining: 12.5s
18:	learn: 0.6465779	total: 234ms	remaining: 12.1s
19:	learn: 0.644

191:	learn: 0.5388830	total: 1.25s	remaining: 5.25s
192:	learn: 0.5387394	total: 1.25s	remaining: 5.24s
193:	learn: 0.5385348	total: 1.26s	remaining: 5.23s
194:	learn: 0.5384096	total: 1.26s	remaining: 5.22s
195:	learn: 0.5381797	total: 1.27s	remaining: 5.21s
196:	learn: 0.5378162	total: 1.27s	remaining: 5.2s
197:	learn: 0.5377201	total: 1.28s	remaining: 5.18s
198:	learn: 0.5374510	total: 1.28s	remaining: 5.17s
199:	learn: 0.5373429	total: 1.29s	remaining: 5.15s
200:	learn: 0.5371829	total: 1.29s	remaining: 5.14s
201:	learn: 0.5369633	total: 1.3s	remaining: 5.12s
202:	learn: 0.5365294	total: 1.3s	remaining: 5.11s
203:	learn: 0.5359015	total: 1.3s	remaining: 5.09s
204:	learn: 0.5355617	total: 1.31s	remaining: 5.07s
205:	learn: 0.5354103	total: 1.31s	remaining: 5.06s
206:	learn: 0.5352717	total: 1.32s	remaining: 5.05s
207:	learn: 0.5351486	total: 1.32s	remaining: 5.03s
208:	learn: 0.5349987	total: 1.33s	remaining: 5.02s
209:	learn: 0.5347411	total: 1.33s	remaining: 5.01s
210:	learn: 0.53

372:	learn: 0.5019872	total: 2.14s	remaining: 3.6s
373:	learn: 0.5018214	total: 2.14s	remaining: 3.59s
374:	learn: 0.5014812	total: 2.15s	remaining: 3.58s
375:	learn: 0.5011828	total: 2.15s	remaining: 3.58s
376:	learn: 0.5008329	total: 2.16s	remaining: 3.57s
377:	learn: 0.5006709	total: 2.16s	remaining: 3.56s
378:	learn: 0.5004818	total: 2.17s	remaining: 3.56s
379:	learn: 0.5002867	total: 2.17s	remaining: 3.55s
380:	learn: 0.5001144	total: 2.18s	remaining: 3.54s
381:	learn: 0.4999515	total: 2.18s	remaining: 3.53s
382:	learn: 0.4997730	total: 2.19s	remaining: 3.52s
383:	learn: 0.4996369	total: 2.19s	remaining: 3.52s
384:	learn: 0.4993862	total: 2.19s	remaining: 3.51s
385:	learn: 0.4992116	total: 2.2s	remaining: 3.5s
386:	learn: 0.4989603	total: 2.2s	remaining: 3.49s
387:	learn: 0.4987788	total: 2.21s	remaining: 3.48s
388:	learn: 0.4984096	total: 2.21s	remaining: 3.48s
389:	learn: 0.4983067	total: 2.22s	remaining: 3.47s
390:	learn: 0.4981494	total: 2.22s	remaining: 3.46s
391:	learn: 0.49

557:	learn: 0.4677137	total: 3.02s	remaining: 2.39s
558:	learn: 0.4675429	total: 3.02s	remaining: 2.38s
559:	learn: 0.4674094	total: 3.03s	remaining: 2.38s
560:	learn: 0.4672679	total: 3.03s	remaining: 2.37s
561:	learn: 0.4670900	total: 3.04s	remaining: 2.37s
562:	learn: 0.4668267	total: 3.04s	remaining: 2.36s
563:	learn: 0.4666946	total: 3.04s	remaining: 2.35s
564:	learn: 0.4665710	total: 3.05s	remaining: 2.35s
565:	learn: 0.4663681	total: 3.05s	remaining: 2.34s
566:	learn: 0.4661851	total: 3.06s	remaining: 2.33s
567:	learn: 0.4660456	total: 3.06s	remaining: 2.33s
568:	learn: 0.4659064	total: 3.06s	remaining: 2.32s
569:	learn: 0.4657580	total: 3.07s	remaining: 2.31s
570:	learn: 0.4656581	total: 3.07s	remaining: 2.31s
571:	learn: 0.4655497	total: 3.08s	remaining: 2.3s
572:	learn: 0.4653711	total: 3.08s	remaining: 2.29s
573:	learn: 0.4652340	total: 3.08s	remaining: 2.29s
574:	learn: 0.4651321	total: 3.09s	remaining: 2.28s
575:	learn: 0.4649802	total: 3.09s	remaining: 2.27s
576:	learn: 0

727:	learn: 0.4452231	total: 3.73s	remaining: 1.39s
728:	learn: 0.4450659	total: 3.73s	remaining: 1.39s
729:	learn: 0.4449153	total: 3.74s	remaining: 1.38s
730:	learn: 0.4447971	total: 3.74s	remaining: 1.38s
731:	learn: 0.4446167	total: 3.75s	remaining: 1.37s
732:	learn: 0.4444528	total: 3.75s	remaining: 1.37s
733:	learn: 0.4443057	total: 3.76s	remaining: 1.36s
734:	learn: 0.4441211	total: 3.76s	remaining: 1.35s
735:	learn: 0.4440573	total: 3.76s	remaining: 1.35s
736:	learn: 0.4439518	total: 3.77s	remaining: 1.34s
737:	learn: 0.4438104	total: 3.77s	remaining: 1.34s
738:	learn: 0.4436747	total: 3.78s	remaining: 1.33s
739:	learn: 0.4435799	total: 3.78s	remaining: 1.33s
740:	learn: 0.4434529	total: 3.78s	remaining: 1.32s
741:	learn: 0.4433691	total: 3.79s	remaining: 1.32s
742:	learn: 0.4432564	total: 3.79s	remaining: 1.31s
743:	learn: 0.4431393	total: 3.8s	remaining: 1.31s
744:	learn: 0.4430202	total: 3.8s	remaining: 1.3s
745:	learn: 0.4429408	total: 3.8s	remaining: 1.29s
746:	learn: 0.44

900:	learn: 0.4248329	total: 4.44s	remaining: 488ms
901:	learn: 0.4247056	total: 4.45s	remaining: 483ms
902:	learn: 0.4246079	total: 4.45s	remaining: 478ms
903:	learn: 0.4245175	total: 4.45s	remaining: 473ms
904:	learn: 0.4244228	total: 4.46s	remaining: 468ms
905:	learn: 0.4243242	total: 4.46s	remaining: 463ms
906:	learn: 0.4242521	total: 4.47s	remaining: 458ms
907:	learn: 0.4241644	total: 4.47s	remaining: 453ms
908:	learn: 0.4240694	total: 4.47s	remaining: 448ms
909:	learn: 0.4239664	total: 4.48s	remaining: 443ms
910:	learn: 0.4238810	total: 4.48s	remaining: 438ms
911:	learn: 0.4237813	total: 4.49s	remaining: 433ms
912:	learn: 0.4236613	total: 4.49s	remaining: 428ms
913:	learn: 0.4235357	total: 4.49s	remaining: 423ms
914:	learn: 0.4234456	total: 4.5s	remaining: 418ms
915:	learn: 0.4233345	total: 4.5s	remaining: 413ms
916:	learn: 0.4232061	total: 4.51s	remaining: 408ms
917:	learn: 0.4230457	total: 4.51s	remaining: 403ms
918:	learn: 0.4228844	total: 4.51s	remaining: 398ms
919:	learn: 0.

In [34]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
tuned_cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.6907298	total: 50.8ms	remaining: 5.03s
1:	learn: 0.6894892	total: 105ms	remaining: 5.16s
2:	learn: 0.6885905	total: 161ms	remaining: 5.21s
3:	learn: 0.6869870	total: 217ms	remaining: 5.2s
4:	learn: 0.6853325	total: 277ms	remaining: 5.26s
5:	learn: 0.6834098	total: 332ms	remaining: 5.2s
6:	learn: 0.6814347	total: 386ms	remaining: 5.12s
7:	learn: 0.6801403	total: 442ms	remaining: 5.08s
8:	learn: 0.6788242	total: 496ms	remaining: 5.02s
9:	learn: 0.6776116	total: 553ms	remaining: 4.98s
10:	learn: 0.6759184	total: 607ms	remaining: 4.91s
11:	learn: 0.6746354	total: 665ms	remaining: 4.87s
12:	learn: 0.6723875	total: 723ms	remaining: 4.84s
13:	learn: 0.6711261	total: 793ms	remaining: 4.87s
14:	learn: 0.6698886	total: 855ms	remaining: 4.85s
15:	learn: 0.6686721	total: 916ms	remaining: 4.81s
16:	learn: 0.6673800	total: 978ms	remaining: 4.77s
17:	learn: 0.6663922	total: 1.03s	remaining: 4.72s
18:	learn: 0.6655287	total: 1.0

68:	learn: 0.6614306	total: 1.4s	remaining: 628ms
69:	learn: 0.6611465	total: 1.42s	remaining: 607ms
70:	learn: 0.6606345	total: 1.44s	remaining: 588ms
71:	learn: 0.6601625	total: 1.46s	remaining: 567ms
72:	learn: 0.6596799	total: 1.48s	remaining: 546ms
73:	learn: 0.6594244	total: 1.5s	remaining: 526ms
74:	learn: 0.6589161	total: 1.51s	remaining: 505ms
75:	learn: 0.6581361	total: 1.53s	remaining: 485ms
76:	learn: 0.6576892	total: 1.55s	remaining: 464ms
77:	learn: 0.6573433	total: 1.57s	remaining: 444ms
78:	learn: 0.6570581	total: 1.6s	remaining: 426ms
79:	learn: 0.6566491	total: 1.63s	remaining: 407ms
80:	learn: 0.6564237	total: 1.65s	remaining: 387ms
81:	learn: 0.6561647	total: 1.67s	remaining: 367ms
82:	learn: 0.6558940	total: 1.69s	remaining: 346ms
83:	learn: 0.6555908	total: 1.71s	remaining: 325ms
84:	learn: 0.6552857	total: 1.73s	remaining: 305ms
85:	learn: 0.6547535	total: 1.75s	remaining: 284ms
86:	learn: 0.6544916	total: 1.77s	remaining: 264ms
87:	learn: 0.6540956	total: 1.78s	

--- Visualisation ---

In [35]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

be_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
be_precision= be_precision.sort_values(by= 'Precision Score', ascending=False)
print(be_precision)

                                    Model  Precision Score
12      Tuned GradientBoosting Precision:         0.422222
13                         XGB Precision:         0.395415
14                   Tuned XGB Precision:         0.386243
2          Tuned Decision Tree Precision:         0.383333
15                        LGBM Precision:         0.379888
7                Random Forest Precision:         0.369942
11            GradientBoosting Precision:         0.358974
3         KNeighborsClassifier Precision:         0.358816
1                Decision Tree Precision:         0.354545
0          Logistic Regression Precision:         0.350398
17                    CatBoost Precision:         0.347826
5                   GaussianNB Precision:         0.343301
8          Tuned Random Forest Precision:         0.342020
18              Tuned CatBoost Precision:         0.339623
6                          SVM Precision:         0.337520
4   Tuned KNeighborsClassifier Precision:         0.3323

In [36]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

be_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
be_accuracy= be_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(be_accuracy)

                                   Model  Accuracy Score
16                  Tuned LGBM Accuracy:        0.638334
11            GradientBoosting Accuracy:        0.635482
12      Tuned GradientBoosting Accuracy:        0.633771
2          Tuned Decision Tree Accuracy:        0.625784
14                   Tuned XGB Accuracy:        0.617228
15                        LGBM Accuracy:        0.617228
10              Tuned AdaBoost Accuracy:        0.617228
17                    CatBoost Accuracy:        0.613805
13                         XGB Accuracy:        0.600114
7                Random Forest Accuracy:        0.590416
8          Tuned Random Forest Accuracy:        0.586423
18              Tuned CatBoost Accuracy:        0.583571
4   Tuned KNeighborsClassifier Accuracy:        0.577296
1                Decision Tree Accuracy:        0.532230
9                     AdaBoost Accuracy:        0.526526
6                          SVM Accuracy:        0.523674
3         KNeighborsClassifier 

In [37]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

be_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
be_auc= be_auc.sort_values(by= 'AUC Score', ascending=False)
print(be_auc)

                              Model  AUC Score
13                         XGB AUC:   0.524967
2          Tuned Decision Tree AUC:   0.512254
14                   Tuned XGB AUC:   0.511660
15                        LGBM AUC:   0.508947
11            GradientBoosting AUC:   0.506536
17                    CatBoost AUC:   0.505439
12      Tuned GradientBoosting AUC:   0.504740
16                  Tuned LGBM AUC:   0.504715
7                Random Forest AUC:   0.500858
8          Tuned Random Forest AUC:   0.499906
3         KNeighborsClassifier AUC:   0.498769
1                Decision Tree AUC:   0.496972
10              Tuned AdaBoost AUC:   0.495646
18              Tuned CatBoost AUC:   0.490058
0          Logistic Regression AUC:   0.486474
6                          SVM AUC:   0.482163
9                     AdaBoost AUC:   0.481712
5                   GaussianNB AUC:   0.480854
4   Tuned KNeighborsClassifier AUC:   0.463609
